# Convolution Neural Networks with GLOVE o WORD2VEC

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../") 

from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn import decomposition
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfVectorizer
)
from sklearn.metrics import f1_score
from ast import literal_eval

from personal_library.sce_keras.callbacks import LearningRateDecay
from personal_library.sce_keras.loss_functions import f1_loss
from personal_library.sce_keras.metrics_functions import f1

/Users/sebastiancorrea/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
data = np.load('../data/numpy_vectorize_data.npy', allow_pickle=True)
data.shape

In [5]:
x = np.load('../data/dataset/X.npy', allow_pickle=True)#.reshape(x.shape[0], x.shape[1], x.shape[2], 1)
y = np.load('../data/dataset/y.npy', allow_pickle=True)

shape = x[0].shape
x_train, x_val, y_train, y_val = train_test_split(x, y, random_state=1992, test_size=0.2)

In [6]:
print("x_train", x_train.shape)
print("y_train", y_train.shape)
print("x_val", x_val.shape)
print("y_val", y_val.shape)

x_train (25569, 19, 300)
y_train (25569,)
x_val (6393, 19, 300)
y_val (6393,)


In [7]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers import (
    Flatten,
    Activation,
    Dense,
    LSTM,
    Dropout,
    SpatialDropout1D,
    Bidirectional,
    GRU, LSTM,
    Conv1D, BatchNormalization,
    Input, GlobalMaxPool1D,MaxPooling1D,
    Add, GlobalAveragePooling1D, GlobalMaxPooling1D,
    concatenate, SpatialDropout1D,   
) 


num_classes = 1
batch_size = 32
epochs = 100
learnRate = 0.00001

lrate_decay = LearningRateDecay(epochs, learnRate).step_decay

checkpoint_path = "../model_wehigts/5_w.hdf5"
checkpointer = ModelCheckpoint(filepath=checkpoint_path,
                               monitor='val_loss', verbose=2,
                               save_best_only=True, mode='min')
units = 128
kernel_size1 = 4
spatial_dr = 0.5
kernel_size2 = 4
dense_units = 64 
dr=0.2
conv_size=32

inp = Input(shape=shape)
x1 = SpatialDropout1D(spatial_dr)(inp)

x_gru = Bidirectional(GRU(units, return_sequences=True))(x1)
x1 = Conv1D(conv_size, kernel_size=kernel_size1, padding='valid', kernel_initializer='he_uniform')(x_gru)
avg_pool1_gru = GlobalAveragePooling1D()(x1)
max_pool1_gru = GlobalMaxPooling1D()(x1)

x3 = Conv1D(conv_size, kernel_size=kernel_size2, padding='valid', kernel_initializer='he_uniform')(x_gru)
avg_pool3_gru = GlobalAveragePooling1D()(x3)
max_pool3_gru = GlobalMaxPooling1D()(x3)

x_lstm = Bidirectional(LSTM(units, return_sequences=True))(x1)
x1 = Conv1D(conv_size, kernel_size=kernel_size1, padding='valid', kernel_initializer='he_uniform')(x_lstm)
avg_pool1_lstm = GlobalAveragePooling1D()(x1)
max_pool1_lstm = GlobalMaxPooling1D()(x1)

x3 = Conv1D(conv_size, kernel_size=kernel_size2, padding='valid', kernel_initializer='he_uniform')(x_lstm)
avg_pool3_lstm = GlobalAveragePooling1D()(x3)
max_pool3_lstm = GlobalMaxPooling1D()(x3)


x = concatenate([avg_pool1_gru, max_pool1_gru, avg_pool3_gru, max_pool3_gru,
                avg_pool1_lstm, max_pool1_lstm, avg_pool3_lstm, max_pool3_lstm])
x = BatchNormalization()(x)
x = Dropout(dr)(Dense(dense_units, activation='relu') (x))
x = BatchNormalization()(x)
x = Dropout(dr)(Dense(int(dense_units / 2), activation='relu') (x))
x = Dense(1, activation = "sigmoid")(x)

model = Model(inputs=inp, outputs=x)
model.summary()
adam = Adam(lr=learnRate, beta_1=0.9, beta_2=0.999,
            epsilon=None, decay=1e-6, amsgrad=False)

# 'binary_crossentropy'
model.compile(loss=f1_loss, 
              optimizer=adam, 
              metrics=['accuracy', f1]) 

history = model.fit(x_train, y_train, 
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=2,
                    validation_data=(x_val, y_val),
                    callbacks=[checkpointer, lrate_decay])

Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 19, 300)      0                                            
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 19, 300)      0           input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 19, 256)      329472      spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 16, 32)       32800       bidirectional_1[0][0]            
__________________________________________________________________________________________________
bidirectio

FailedPreconditionError: Attempting to use uninitialized value batch_normalization_1/moving_variance/local_step
	 [[Node: batch_normalization_1/AssignMovingAvg_1/batch_normalization_1/moving_variance/AssignAdd = AssignAdd[T=DT_FLOAT, _class=["loc:@batch_normalization_1/AssignMovingAvg_1"], use_locking=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](batch_normalization_1/moving_variance/local_step, batch_normalization_1/AssignMovingAvg_1/batch_normalization_1/moving_variance/AssignAdd/value)]]

In [ ]:
#Load best model
model.load_weights(checkpoint_path)
y_pred = model.predict(x_val, batch_size=1)
y_pred = np.where(y_pred > 0.5, 1, 0)

print("Neural Network f1_sklearn: {}".format(f1_score(y_val, y_pred)))